In [1]:
#############################################
##    Train-Test Algorithm Test Harness    ##
#############################################

from random import seed, randrange
from csv import reader


# Load csv file 
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [2]:
# Convert string to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())


In [8]:
# Split a dataset into a train and test sets
def train_test_split(dataset, split):
    train = list()
    train_size = split * len(dataset)
    dataset_copy = list(dataset)
    while len(train) < train_size:
        index = randrange(len(dataset_copy))
        train.append(dataset_copy.pop(index))
    return train, dataset_copy


In [4]:
# Calculate accuracy percentage    
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


In [5]:
# Train-Test Split to evaluate an algorithm
def evaluate_algorithm(dataset, algorithm, split, *args):
    train, test = train_test_split(dataset, split)
    test_set = list()
    for row in test:
        row_copy = list(row)
        row_copy[-1] = None
        test_set.append(row_copy)
    predicted = algorithm(train, test_set, *args)
    actual = [row[-1] for row in test]
    accuracy = accuracy_metric(actual, predicted)
    return accuracy


In [6]:
# Baseline: Zero-Rule Algorithm
def zero_rule_algorithm_classification(train, test):
    output_values = [row[-1] for row in train]
    prediction = max(set(output_values), key=output_values.count)
    predicted = [prediction for i in range(len(test))]
    return predicted


In [17]:
seed(1)
filename = 'pima-indians-diabetes.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
    
split = 0.6
accuracy = evaluate_algorithm(dataset, 
                              zero_rule_algorithm_classification, 
                              split
                             )

print("Accuracy: %.3f%%" % (accuracy))


Accuracy: 67.427%


In [26]:
# k-folds split

def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
    

In [24]:
## Cross Validation Split Algorithm Evaluation

def evaluate_algorithm_cv(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores


In [27]:
seed(1)
filename = 'pima-indians-diabetes.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
    
# Evaluate Algorithm 
n_folds = 5
scores = evaluate_algorithm_cv(dataset, 
                               zero_rule_algorithm_classification, 
                               n_folds
                              )

print("Scores: %s" % scores)
print("Mean Accuracy: %.3f%%" % (sum(scores) / len(scores)))



Scores: [62.091503267973856, 64.70588235294117, 64.70588235294117, 64.70588235294117, 69.28104575163398]
Mean Accuracy: 65.098%
